## A Mechanism-aware and Multi-omic Machine Learning Pipeline Characterises Yeast Cell Growth 

**Deep Learning Modelling Codebase**




We begin with importing the necessary files, we use Keras with Tensorflow 2.0 and the Shap Library for analysing feature importance

In [ ]:
import pyreadr
import csv
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pa
import os.path
import matplotlib.pyplot as plts
import seaborn as sns
from tensorflow.keras import backend as F
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Concatenate, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import max_norm
from sklearn import preprocessing
from tensorflow.keras.utils import plot_model
import pandas as pd 
import shap
from sklearn.linear_model import LinearRegression

SEED = 120 # fixed for reproducability


In [ ]:
#Checks that the reading of the data has worked
def check_file_read_ok(data, name):
    if data is None:
        print("error with " + name + " data not read")
    else:
        print("Success in loading " + name)
        print(data.shape)

def remove_zero_entry_columns(data):
    return data.loc[:, (data != 0).any(axis = 0)]

In [ ]:

#Name of the row we are trying to predict (growth rate)
TARGET_NAME = 'log2relT'


file_loc = ''

with open(file_loc + 'testing_index.csv', 'r') as csvfile:
    testing_index = []
    for row in csv.reader(csvfile, delimiter=';'):
        testing_index.append(row[0]) # careful here with [0]
        
        

We next import all the datasets, these are a mix of: 




*   **expression_data** - refers to the msb145172 dataset (Duibhir et al., 2014) containing transcription rates along with accompanying growth rates, 'log2relT' variable.  
*   **metabolic_expression_data** - refers to a subset of the expression data which are used in a metabolic modelling stage
* **reaction_data** - refers to the set of reaction flux rates after feeding in the expression data into a flux balance analyse optimised yeast metabolic model 
*  **iRF** - refers to the a subset of the features (both expression and reaction) that are selected for importance using the Iterative Random Forest algorithm 
* **SGL** - refers to the selected features selected by the sparse-group-lasso feature selection algorithm 
* **Gens** - refers to features selected by the NSGA-II algorithm 
* **independent_data** - refers to an experimentally independent test set captured in a environment that closely resembles the msb expression data and is put through metabolic modelling to create an independent set to test generalisability 
* **full_data** - contains the expression data and metabolic expression data combined




In [ ]:
#Load the data removing any
full_data = pyreadr.read_r('data/completeDataset.RDS')[None]
check_file_read_ok(full_data, "full data")

# Here we extract the features highlighted by the genetic algorithm
def get_the_genetic_algo_data():
    gens = []
    features = pa.read_csv(file_loc + "data/genetic_feature_selection_features.csv", header = None)
    for x in range(features.shape[0]):
        fet = features.iloc[x, :].values
        gens.append(full_data[fet])

    return gens

gens = get_the_genetic_algo_data()

full_data = remove_zero_entry_columns(full_data)
expression_data = pyreadr.read_r('data/expressionOnly.RDS')[None]

check_file_read_ok(expression_data, "expression data")
expression_data = remove_zero_entry_columns(expression_data)


metabolic_expression_data = pyreadr.read_r('data/metabolic_gene_data.RDS')[None]
check_file_read_ok(metabolic_expression_data, "met_expression data")

#Extract the target and drop target column from main data
target_data = full_data[TARGET_NAME]
full_data = full_data.drop(columns=TARGET_NAME)
expression_data = expression_data.drop(columns=TARGET_NAME)


iRF = pa.read_csv(file_loc + 'data/Features_Extracted_Using_iRF.csv', header = None)
iRF.columns = ['Genes']
check_file_read_ok(iRF, "iRF data")
iRF = remove_zero_entry_columns(iRF)
iRF = full_data[iRF.Genes]

#SGL feature extracted data 
sgl = pa.read_csv(file_loc + 'data/Features_Extracted_Using_SGL.csv', header = None)
sgl.columns = ['Genes']
check_file_read_ok(sgl, "sgl data")
sgl_list= []
for x in sgl.Genes:
    if x in full_data.columns:
        sgl_list.append(x)   #We do this because some of the zero features have been removed from full_data

sgl = full_data[sgl_list]
knockouts = full_data['Row']

full_data = full_data.drop(columns = 'Row')
reaction_data = full_data.drop(columns = expression_data.columns.values)

# Independent dataset
independent_data = pd.read_csv(file_loc + 'data/completeDataset_ITS_gamma1.csv')
independent_knockouts = independent_data['Row']

### Imputting missing values in the independent set 


We next look to impute any missing features in the experimental independent test set 

In [ ]:
# We need to impute the columns that are missing from the independent dataset 
to_be_predicted = list(set(full_data.columns) - set(independent_data.columns))
# There are a couple in the independent that are not in our full dataset 
to_be_dropped_from_independent = list(set(independent_data.columns) - set(full_data.columns))


independent_data = independent_data.drop(to_be_dropped_from_independent, axis = 1)
targets_impute  = full_data[to_be_predicted]

original_data_train = full_data.drop(to_be_predicted, axis = 1)
data_to_be_predicted = independent_data.copy() 


We apply linear regression to predict the missing features using the original full (msb) dataset as training 

In [ ]:
for x in range(targets_impute.shape[1]):
    lm = LinearRegression()
    lm.fit(original_data_train, targets_impute.iloc[:, x])
    predictions = lm.predict(original_data_train)
    predicted_values = lm.predict(data_to_be_predicted)
    independent_data[targets_impute.columns[x]] = predicted_values 

We remove any duplicate gene knockouts that are already in the full dataset 

In [ ]:
new_rows = list(set(independent_knockouts) - set(knockouts))
independent_lr = independent_data[independent_knockouts.isin(new_rows)]


We later will use this experimentally independent data in varifying the generalisability of the models 

In [ ]:
#Independent data with linear regression imputting the missing values 

independent_target = pd.read_csv(file_loc + 'data/independent_target.csv')
independent_genes = independent_knockouts

def split_expression_and_fluxes(data):
  return data[expression_data.columns], data[reaction_data.columns]

independent_lr_expression, independent_lr_fluxes = split_expression_and_fluxes(independent_lr)


We build a 3 layer feed-forward network using Keras for the single-view models and a multi-modal model which using a concatenation layer to join two transfer learned single-view models 

In [ ]:

def init_model_3layer(input_dim, learning_rate, epochs, momentum, neurons):
    model = Sequential()
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3)))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3)))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(neurons, input_dim = input_dim, kernel_constraint=max_norm(3)))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(1))
    model.add(Activation('linear'))
    rms = SGD(lr= learning_rate, decay= learning_rate / epochs, momentum=momentum)
    model.compile(loss='mean_absolute_error', optimizer=rms, metrics = ["mean_absolute_error", 'mean_squared_error'])
    return model

def init_model(input_dim, learning_rate, epochs, momentum, neurons, trainable = True):
    input = Input(shape = (input_dim,))
    layer = Dense(neurons, activation='sigmoid', kernel_constraint=max_norm(3), name = "expression_1") (input)
    layer = Dropout(rate=0.4) (layer)
    layer = Dense(neurons, activation='sigmoid', kernel_constraint=max_norm(3), name = "expression_2") (layer)
    layer = Dropout(rate=0.4) (layer)
    predictions = Dense(1, activation='linear') (layer)
    model = Model(inputs = input, outputs = predictions)
    rms = SGD(lr= learning_rate, decay= learning_rate / epochs, momentum=momentum)
    model.trainable = trainable
    if (trainable) :
        model.compile(loss='mean_squared_error', optimizer=rms, metrics = ["mean_absolute_error"])
    return model

def init_multi_model(input_dim,input_dim2, learning_rate, epochs, momentum, neurons, reaction_trained, expression_trained):
    reaction_input = Input(shape = (input_dim,))
    expression_input = Input(shape = (input_dim2,))
    comb_layer = Concatenate()([reaction_trained(reaction_input), expression_trained(expression_input)])
    comb_layer = Dense(neurons, activation='sigmoid', kernel_constraint=max_norm(3), name = "last_hidden") (comb_layer)
    predictions = Dense(1, activation='linear') (comb_layer)
    model = Model(inputs = [reaction_input,expression_input], outputs = predictions)
    rms = SGD(lr= learning_rate, decay= learning_rate / epochs, momentum=momentum)
    model.compile(loss='mean_squared_error', optimizer=rms, metrics = ["mean_absolute_error"])
    return model



Parameter setting

In [ ]:

SEED = 120 # we fix the seed for reproducability 
number_of_instances = len(target_data) # how many datapoints do we have 
testing_index = list(map(int, testing_index)) # fixed testing indexes
training_index = np.setxor1d(range(1,number_of_instances), testing_index) # not testing index 
epochs = 1000 # how many epochs we run the network for in training 
batches = 256 # the batch size in training 
momentum = 0.75 
lrate = 0.005 
neurons = 1000 # how many neurons in each hidden unit
validation = 0.1 # we use a small validation set due to restricted dataset size

Normalising and general readying of the data

In [ ]:

target_data_train, target_data_test = target_data.drop(target_data.index[testing_index]), target_data.iloc[testing_index]
target_data_test.to_csv(file_loc + 'data/test_growth_target.csv')


#Split the data 80:20
full_data_train, full_data_test = full_data.drop(full_data.index[testing_index]), full_data.iloc[testing_index, :]
expression_data_train, expression_data_test = expression_data.drop(expression_data.index[testing_index]), expression_data.iloc[testing_index, :]
metabolic_expression_data_train, metabolic_expression_data_test = metabolic_expression_data.drop(metabolic_expression_data.index[testing_index]), metabolic_expression_data.iloc[testing_index, :]
reaction_data_train, reaction_data_test = reaction_data.drop(reaction_data.index[testing_index]), reaction_data.iloc[testing_index, :]
target_data_train, target_data_test = target_data.drop(target_data.index[testing_index]), target_data.iloc[testing_index]
iRF_train, iRF_test = iRF.drop(iRF.index[testing_index]), iRF.iloc[testing_index, :]
sgl_train, sgl_test = sgl.drop(sgl.index[testing_index]), sgl.iloc[testing_index, :]


#Preprocessing the data to mean of zero and unit variance - stopped using this for improved results
full_scaler = preprocessing.StandardScaler().fit(full_data_train)
full_data_scaled_train = full_scaler.transform(full_data_train).astype(np.float32)
full_data_scaled_test = full_scaler.transform(full_data_test).astype(np.float32)

expression_scaler = preprocessing.StandardScaler().fit(expression_data_train)
expression_data_scaled_train = expression_scaler.transform(expression_data_train).astype(np.float32)
expression_data_scaled_test = expression_scaler.transform(expression_data_test).astype(np.float32)
independent_lr_expression_scaled = expression_scaler.transform(independent_lr_expression).astype(np.float32)
np.savetxt(file_loc + 'data/independent_lr_expression_scaled.csv', independent_lr_expression_scaled, delimiter=",") # we save the independent test set to be used by the R code 


reaction_scaler = preprocessing.StandardScaler().fit(reaction_data_train)
reaction_data_scaled_train = reaction_scaler.transform(reaction_data_train).astype(np.float32)
reaction_data_scaled_test = reaction_scaler.transform(reaction_data_test).astype(np.float32)
independent_lr_reaction_scaled = reaction_scaler.transform(independent_lr_fluxes).astype(np.float32)


metabolic_expression_scaler = preprocessing.StandardScaler().fit(metabolic_expression_data_train)
metabolic_expression_data_scaled_train = metabolic_expression_scaler.transform(metabolic_expression_data_train).astype(np.float32)
metabolic_expression_data_scaled_test = metabolic_expression_scaler.transform(metabolic_expression_data_test).astype(np.float32)

iRF_scaler = preprocessing.StandardScaler().fit(iRF_train)
iRF_scaled_train = iRF_scaler.transform(iRF_train).astype(np.float32)
iRF_scaled_test = iRF_scaler.transform(iRF_test).astype(np.float32)

sgl_scale = preprocessing.StandardScaler().fit(sgl_train)
sgl_scaled_train = sgl_scale.transform(sgl_train).astype(np.float32)
sgl_scaled_test = sgl_scale.transform(sgl_test).astype(np.float32)

target_train = target_data_train.astype(np.float32)
target_test = target_data_test.astype(np.float32)


Here we define the training experiments and the method to extract out the pre-trained model to be joined in a concatenation layer for the multi-modal experiments 

In [ ]:
def run_experiment_full(name, index_in, data_train, data_test,requires_weight_save = False, requires_prediction_save = True, verbose = False):
  model_full = init_model(data_train.shape[1], lrate,epochs, momentum, 1000)
  if requires_weight_save:
    if os.path.exists(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5'): # should we load the weights? 
      model_full.load_weights(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5')
    else:
      model_full.fit(x = data_train, y = target_train, epochs=epochs, batch_size=batches, validation_split=validation, verbose=verbose)
      model_full.save_weights(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5') # we save the weights
  else: 
    model_full.fit(x = data_train, y = target_train, epochs=epochs, batch_size=batches, validation_split=validation, verbose=verbose)
  score = model_full.evaluate(data_test, target_test, verbose=1)
  if requires_prediction_save:
  #  if not os.path.exists(file_loc +'predictions/'+name+'_Predictions_' + str(index_in) + '.csv'): # do we already have a prediction set 
       prediction = model_full.predict_on_batch(data_test)
       np.savetxt(fname=file_loc +'predictions/'+name+'_Predictions_' + str(index_in) + ".csv",  X=prediction, delimiter=',') # save the prediction set 
  return score

# We remove the final layer from the pre-trained model and re-compile to allow it to be joined in a layer layer 
def load_older_model(data_train, fname, clipped = True):
  model = init_model(data_train.shape[1], lrate,epochs, momentum, neurons)
  model.load_weights(file_loc + 'models/'+fname+'_model.h5')
  rms = SGD(lr= lrate, decay= lrate / epochs, momentum=momentum)
  model.compile(loss='mean_squared_error', optimizer=rms, metrics = ["mean_absolute_error"])
  model.summary()
  if clipped:
   model.trainable = True
   model._layers.pop()
   model._layers.pop()
   model.outputs = [model.layers[-1].output]
   model._layers[-1].output_nodes = []
   model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
   model.compile(loss='mean_squared_error', optimizer=rms, metrics = ["mean_absolute_error"])
   model.summary()
  else:
    return model
  return model 

weights_model_run = []
outputs = [] 

# The multi-modal model is used as a way of integrating two separate transfer learned view-point - fluxes and expressions 
def run_experiment_mm_full(name, index_in, data_train_1, data_train_2, data_test_1, data_test_2, name_1, name_2, requires_weight_save = False, requires_prediction_save = True, verbose = False):
  model_1 = load_older_model(data_train_1, name_1)
  model_2 = load_older_model(data_train_2, name_2)
  model_full = init_multi_model(data_train_1.shape[1], data_train_2.shape[1], lrate, epochs, 0.75, 15, model_1, model_2)
  if requires_weight_save:
    if os.path.exists(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5'):
      model_full.load_weights(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5')
    else:
      model_full.fit(x = [data_train_1, data_train_2], y = target_train, epochs=epochs, batch_size=batches, validation_split=validation,  verbose=verbose)
      model_full.save_weights(file_loc + 'models/'+name+'_model' + str(index_in) + '.h5')
  else: 
    model_full.fit(x = [data_train_1, data_train_2], y = target_train, epochs=epochs, batch_size=batches, validation_split=validation, verbose=verbose)
  score = model_full.evaluate([data_test_1, data_test_2], target_test, verbose=1)
  if requires_prediction_save:
    if not os.path.exists(file_loc +'predictions/'+name+'_Predictions_' + str(index_in) + '.csv'):
       prediction = model_full.predict_on_batch([data_test_1, data_test_2])
       np.savetxt(fname=file_loc +'predictions/'+name+'_Predictions_' + str(index_in) + ".csv",  X=prediction, delimiter=',')
  if name_2 == 'expression':
    weights_model_run.append(model_full.get_layer(name = "last_hidden").get_weights()[0])
    outputs.append(model_full.layers[-1].get_weights())
  return score


We run each set of experiments for 100 runs so that we can gain confidence scores

In [ ]:
def run_nsga(): 
  for i in range(0,10):
    for x in range(0,9):
          next = gens[5]
          next_train, next_test = next.drop(next.index[testing_index]), next.iloc[testing_index, :]
          scale = preprocessing.StandardScaler().fit(next_train)
          next_scale_train = scale.transform(next_train).astype(np.float32)
          next_scale_test = scale.transform(next_test).astype(np.float32)

          score = run_experiment_full('NSGA-II', i*10+x, next_scale_train, next_scale_test)
          print(score)

def run_expression():
  for i in range(0, 101):
    score = run_experiment_full('expression', i, expression_data_scaled_train, expression_data_scaled_test)
    print(score)

#GEM
def run_metabolic():
  for i in range(0,101):
    score = run_experiment_full('metabolic_expression', i, metabolic_expression_data_scaled_train, metabolic_expression_data_scaled_test)
    print(score)

def run_fluxes():
  for i in range(0,101):
    score = run_experiment_full('fluxes', i, reaction_data_scaled_train, reaction_data_scaled_test)
    print(score)

def run_concate_flu_ge():
  for i in range(0,101):
    score = run_experiment_full('concate_Flu_GE', i, full_data_scaled_train, full_data_scaled_test)
    print(score)

def run_irf():
  for i in range(0,101):
    score = run_experiment_full('iRF', i, iRF_scaled_train, iRF_scaled_test)
    print(score)

def run_sgl():
  for i in range(0,101):
    
    score = run_experiment_full('SGL', i, sgl_scaled_train, sgl_scaled_test)
    print(score)


def run_mm_metabolic():
  for i in range(0,101):
    score = run_experiment_mm_full('multi_model_metabolic_expression', i, reaction_data_scaled_train, 
                       metabolic_expression_data_scaled_train, reaction_data_scaled_test, 
                       metabolic_expression_data_scaled_test, 'reaction', 'metabolic_expression')
    print(score)

def run_mm_full():
  for i in range(0,101):
    score = run_experiment_mm_full('multi_model_full_expression', i, reaction_data_scaled_train, 
                       expression_data_scaled_train, reaction_data_scaled_test, 
                       expression_data_scaled_test, 'reaction', 'expression', verbose = False)
    print(score)



# Training all of the models 

In [ ]:
run_expression()
run_fluxes()
run_mm_full()
run_mm_metabolic()
run_sgl()
run_irf()
run_concate_flu_ge()
run_nsga()
run_metabolic()

# Testing on the independent test set

In [ ]:
# Here load in the multi-modal model 

expression_side = load_older_model(independent_lr_expression, 'expression')
flux_side = load_older_model(independent_lr_fluxes, 'reaction')

multi_modal_independent = init_multi_model(independent_lr_fluxes.shape[1], independent_lr_expression.shape[1], lrate, epochs, 0.75, 15, flux_side, expression_side)
multi_modal_independent.load_weights("models/MM-Flu_GE.h5")

# Predict on the independent set multi-modal
independent_predictions_lr_mm = multi_modal_independent.predict([ independent_lr_reaction_scaled, independent_lr_expression_scaled])

# Predict on independent set expression only 
expression_model = load_older_model(independent_lr_expression, 'expression', clipped = False)
independent_predictions_lr_e_dl = expression_model.predict(independent_lr_expression_scaled)

checking = multi_modal_independent.predict([reaction_data_scaled_test, expression_data_scaled_test])
np.savetxt('predictions/Independent/independent_lr_multi_modal_expression_fluxes.csv', independent_predictions_lr_mm, delimiter=",")
np.savetxt('predictions/Independent/independent_lr_dl_expression_only.csv', independent_predictions_lr_e_dl, delimiter=",")

double_gene_knockouts = independent_genes.apply(lambda x : '_' in x)
np.savetxt('data/is_double_gene_knockout.csv', double_gene_knockouts, delimiter=',')

# Calculating the feature importance using the SHAP method 

In [ ]:
# Load the previously saved model 
multi_modal = load_model("models/MM-Flu_GE_n.h5")

# create the background set from which the mean can be generated
background = [reaction_data_scaled_train, expression_data_scaled_train]

# train the explainer 
explainer = shap.DeepExplainer(multi_modal, background)


We calculate a normalised error for each data point so that we can determine which feature importance emphasis cause the issues

In [ ]:
predictions = multi_modal.predict([reaction_data_scaled_test, expression_data_scaled_test])
errors = np.abs(predictions.reshape(228,) - target_test)
norm_errors = (errors - np.mean(errors)) / np.std(errors)


Shap value calculation to gather feature importance for each of the inputs which export to csv format

In [ ]:
test_all = [reaction_data_scaled_test, expression_data_scaled_test]
shap_values_all = explainer.shap_values(test_all)

reaction_shap_values = pd.DataFrame(data = shap_values_all[0][0], columns = reaction_data.columns)
reaction_shap_values['target'] = target_test.values
reaction_shap_values['Normalised_Errors'] = norm_errors.values
expression_shap_values = pd.DataFrame(data = shap_values_all[0][1], columns = expression_data.columns)
expression_shap_values['Normalised_Errors'] = norm_errors.values
expression_shap_values['target'] = target_test.values

reaction_shap_values.to_csv('data/reaction_shap_values.csv')
expression_shap_values.to_csv('data/expression_shap_values.csv')